In [1]:
%pip install torch
%pip install PathLib
%pip install nltk
%pip install wakepy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import torch
from torch import nn, Tensor, FloatTensor
import torch.nn.functional as F

In [7]:
import os
import pickle
import re
from collections import Counter

import nltk
import numpy as np
import torch
import pathlib

In [8]:
DATADIR = 'data'
MOD_VAR_DIR = 'models_and_variables'
TRAINED_MODELS_DIR = 'trained_models'
HP_TEXT_DIR = os.path.join(DATADIR, 'harry_potter_text')
dirs=[MOD_VAR_DIR, TRAINED_MODELS_DIR]
for dir in dirs:
    pathlib.Path(dir).mkdir(parents=True, exist_ok=True)

In [5]:
"""def generate_vocabulary(individual_words, include_special_tokens=False):
    condition_keys = sorted(individual_words)
    print(conditions.unique())
    result = dict(zip(condition_keys, range(len(condition_keys))))
    print(len(result))
    return result
"""

'def generate_vocabulary(individual_words, include_special_tokens=False):\n    condition_keys = sorted(individual_words)\n    print(conditions.unique())\n    result = dict(zip(condition_keys, range(len(condition_keys))))\n    print(len(result))\n    return result\n'

In [9]:
def generate_vocabulary(individual_words, min_threshold, include_special_tokens=False):
    """
    Return {token: index} for all train tokens (words) that occur min_threshold times or more,
        `index` should be from 0 to N, where N is a number of unique tokens in the resulting dictionary.
    """
    #create a list of words that happen min_threshold times or more in that string
    condition_keys = sorted([key for key, value in Counter(individual_words).items() if value >= min_threshold])
    #generate the vocabulary(dictionary)


    if not include_special_tokens:
        result = dict(zip(condition_keys, range(len(condition_keys))))
        return result
    else:
        result = dict(zip(condition_keys, range(3,len(condition_keys)+3)))
        orig = {"BOS": 0, "EOS": 1, "UNK": 2}
        orig.update(result)
        return orig

In [10]:
def get_hp_text(num_of_books=1):
    text_files = os.listdir(HP_TEXT_DIR)
    path_to_hp_text = os.path.join(MOD_VAR_DIR, "harry_potter_text.pkl")

    filepath = pathlib.PurePath(MOD_VAR_DIR, "harry_potter_text.pkl")

    counter=0
    if not os.path.exists(path_to_hp_text):
        all_text = ""
        for book in text_files:
             path_to_book = os.path.join(HP_TEXT_DIR, book)

             with open(path_to_book, "r", encoding="utf8") as f:
                text = f.readlines()

             text = [line for line in text if "Page" not in line]
             text = " ".join(text).replace("\n", "")
             text = [word for word in text.split(" ") if len(word) > 0]

             text = " ".join(text)
             text = re.sub("[^a-zA-Z0-9-_*.!,? \"\']", "", text)
             all_text+=text
             counter+=1
             if counter==num_of_books:
                 break

        with open(path_to_hp_text, 'wb') as handle:
            pickle.dump(all_text, handle, protocol=pickle.HIGHEST_PROTOCOL)
    else:
        with open(path_to_hp_text, 'rb') as handle:
            all_text = pickle.load(handle)

    return all_text

In [11]:
def get_tokens():
    path_to_tokens = os.path.join(MOD_VAR_DIR, "harry_potter_tokens.pkl")
    if not os.path.exists(path_to_tokens):
        tokens=nltk.word_tokenize(get_hp_text())
        with open(path_to_tokens, 'wb') as handle:
            pickle.dump(tokens, handle, protocol=pickle.HIGHEST_PROTOCOL)
    else:
        with open(path_to_tokens, 'rb') as handle:
            tokens = pickle.load(handle)
    return tokens

In [12]:
def get_2_vocabs():
    path_to_vocab= os.path.join(MOD_VAR_DIR, "harry_potter_vocab.pkl")
    path_to_inv_vocab = os.path.join(MOD_VAR_DIR, "harry_potter_inv_vocab.pkl")
    tokens= get_tokens()
    if not os.path.exists(path_to_vocab):
        vocab=generate_vocabulary(tokens, 1)
        inv_vocab = {v: k for k, v in vocab.items()}

        with open(path_to_vocab, 'wb') as handle:
            pickle.dump(vocab, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open(path_to_inv_vocab, 'wb') as handle:
            pickle.dump(inv_vocab, handle, protocol=pickle.HIGHEST_PROTOCOL)
    else:
        with open(path_to_vocab, 'rb') as handle:
             vocab= pickle.load(handle)
        with open(path_to_inv_vocab, 'rb') as handle:
             inv_vocab= pickle.load(handle)

    return vocab, inv_vocab

In [14]:
class DataBuilder:
    def __init__(self, seq_len, tokens, word2idx={}, idx2word={}):
        self.seq_len = seq_len
        self.tokens = tokens
        self.number_of_tokens = len(tokens)
        self.char2idx=word2idx
        self.idx2char=idx2word

    def grab_random_sample(self):
        start = np.random.randint(0, self.number_of_tokens - self.seq_len)
        end = start + self.seq_len
        text_slice = self.tokens[start:end]

        input_text = text_slice[:-1]
        label = text_slice[1:]
        input_text = torch.tensor([self.char2idx[c] for c in input_text], dtype=torch.int32)
        label = torch.tensor([self.char2idx[c] for c in label], dtype=torch.int32)

        return input_text, label

    def grab_random_batch(self, batch_size):
        input_texts, labels = [], []

        for _ in range(batch_size):
            input_text, label = self.grab_random_sample()

            input_texts.append(input_text)
            labels.append(label)

        input_texts = torch.stack(input_texts)
        labels = torch.stack(labels)

        return input_texts, labels

In [15]:
nltk.download('punkt_tab')
import numpy as np
import torch
from torch import nn, Tensor, FloatTensor
import torch.nn.functional as F

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\wilfr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [16]:
class LSTMForWordGeneration(nn.Module):
    def __init__(self, word2idx, idx2word,embedding_dim=128, hidden_size=256, n_layers=3):
        super().__init__()
        self.word2idx = word2idx
        self.idx2word = idx2word
        self.embedding_dim = embedding_dim
        self.num_words = len(word2idx)
        self.hidden_size = hidden_size
        self.n_layers = n_layers

        self.embedding = nn.Embedding(self.num_words, embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim,
                            hidden_size=hidden_size,
                            num_layers=n_layers,
                            batch_first=True)

        self.fc = nn.Linear(hidden_size, self.num_words)

        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x = self.embedding(x)
        output, (h, c) = self.lstm(x)
        logits = self.fc(output)
        return logits

    def write(self, text, max_words, greedy=False):

        idx = torch.tensor([self.word2idx[w] for w in text])
        hidden = torch.zeros(self.n_layers, self.hidden_size)
        cell = torch.zeros(self.n_layers, self.hidden_size)

        for i in range(max_words):

            if i == 0:
                selected_idx = idx
            else:
                selected_idx = idx[-1].unsqueeze(0)

            x = self.embedding(selected_idx)
            out, (hidden, cell) = self.lstm(x, (hidden, cell))

            out = self.fc(out)

            if len(out) > 1:
                out = out[-1, :].unsqueeze(0)

            probs = self.softmax(out)

            if greedy:
                idx_next = torch.argmax(probs).squeeze(0)
            else:
                idx_next = torch.multinomial(probs, num_samples=1).squeeze(0)

            idx = torch.cat([idx, idx_next])

        gen_string = [self.idx2word[int(w)] for w in idx]
        gen_string = " ".join(gen_string)

        return gen_string

In [17]:
from pathlib import Path
from torch import optim, nn
import pandas as pd

def train(model, word2idx, idx2word, tokens, config):
    training_data = {
        "model":[],
        "iteration":[],
        "training data length":[],
        "loss": [],
        "generated text": []
    }
    name=model.__class__.__name__
    new_dir =Path(os.path.join(TRAINED_MODELS_DIR, name))
    new_dir.mkdir(parents=True, exist_ok=True)
    iterations = config["iterations"]
    max_len = config["max_len"]
    evaluate_interval = config["evaluate_interval"]
    embedding_dim = config["embedding_dim"]
    hidden_size = config["hidden_size"]
    n_layers = config["n_layers"]
    lr = config["lr"]
    batch_size = config["batch_size"]
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()
    dataset = DataBuilder(max_len, tokens, word2idx, idx2word)
    model.train()
    for iteration in range(iterations):
        input_texts, labels = dataset.grab_random_batch(batch_size=batch_size)
        input_texts, labels = input_texts, labels
        optimizer.zero_grad()
        output = model(input_texts)
        output = output.transpose(1,2)

        loss = loss_fn(output, labels.long())

        loss.backward()
        optimizer.step()

        if iteration % evaluate_interval == 0:
            model.eval()
            torch.no_grad()
            print("--------------------------------------")
            print(f"training text length {max_len}")
            print(f"Iteration {iteration}")
            print(f"Loss {loss.item()}")
            generated_text = model.write(["Spells"], max_words=50)
            print(generated_text)
            print("--------------------------------------")
            training_data["model"].append(name)
            training_data["iteration"].append(iteration)
            training_data["training data length"].append(max_len)
            training_data["loss"].append(loss.item())
            training_data["generated text"].append(generated_text)
            torch.enable_grad()
            model.train()

    training_data=pd.DataFrame(training_data)
    training_data.set_index(["model", "iteration"])
    training_data.to_csv(os.path.join(TRAINED_MODELS_DIR,name,"training_data.csv"), encoding="utf8")
    config_data=pd.DataFrame(list(config.items()), columns=["key", "value"])
    config_data.to_csv(os.path.join(TRAINED_MODELS_DIR,name,"config_data.csv"), encoding="utf8", index=False)
    torch.save(model.state_dict(), os.path.join(TRAINED_MODELS_DIR,name+"/model_state.pth"))


In [ ]:
def get_config():
    config={}
    config["iterations"]=300
    config["max_len"]=20
    config["evaluate_interval"]=30
    config["embedding_dim"]=128
    config["hidden_size"]=256
    config["n_layers"]=2
    config["lr"]=0.003
    config["batch_size"]=64
    config["bidirectional"]=False
    return config

In [32]:
text = get_hp_text()
tokens = get_tokens()
word2idx, idx2word = get_2_vocabs()


In [ ]:
from wakepy import keep 
config=get_config()
model = LSTMForWordGeneration(word2idx, idx2word, embedding_dim=config["embedding_dim"],
                                      hidden_size=config["hidden_size"], n_layers=config["n_layers"])
with keep.running():
    train(model, word2idx, idx2word, tokens, config)


--------------------------------------
training text length 20
Iteration 0
Loss 8.83820629119873
Spells right-handed with Percy promised Platform Having secret easier trash hanger swiftly ankles Scared Jordans coins wrong cabbages passed standard stopped jinxing collar everything badge forgotten moleskin Wheres VANASHIG constrictors nearby care Baruffio stick folded steeling mustve armed Remembrall Tut swollen stick lion-fish GET heels Fortunately crowds horribly shouted hut escape
--------------------------------------
--------------------------------------
training text length 20
Iteration 30
Loss 6.69385290145874
Spells hes Hogwarts of theyre in sort barrier the join , is during Ive high even into course want clicked at of empty Can . the woke of I him about But starting two never nothing kept Going us , the of , , said one crumpled there you , hear
--------------------------------------
--------------------------------------
training text length 20
Iteration 60
Loss 6.5342712402343

## Attention

To get an idea of what attention means mathematically let's compare some matrix multiplications. To get an idea of what attention means mathematically let's compare some matrix multiplications. These calculations are a means of expressing the mechanisms behind attention. Most of the time q(uery) and k(ey) matrices will contain much more complexity than expressed by the matrix multiplications below.

### Some direct multiplications

Firstly, a relatively straight matrix multiplication of three matrices. In this case the third element of the q(uery) matrix is a 1, all the rest are zeroes. Matrix multiplying q with the one hot encoded k(eys) selects the third row of this k(eys) matrix. As a consequence the adapted k(eys) matrix selects the third v(alues) row.

In [28]:
q = F.one_hot(torch.tensor([2]), 5)
k = F.one_hot(torch.arange(5), 5)
v = torch.randint(2, 8, (5,2 ))
print("q")
print(q)
print("k")
print(k)
print("v")
print(v)

q_times_k=torch.mm(q,k.T)
print("q_times_k")
print(q_times_k)
print("q_times_k_times_v")
print(torch.mm(q_times_k, v))

q
tensor([[0, 0, 1, 0, 0]])
k
tensor([[1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1]])
v
tensor([[7, 2],
        [2, 5],
        [3, 3],
        [6, 2],
        [7, 3]])
q_times_k
tensor([[0, 0, 1, 0, 0]])
q_times_k_times_v
tensor([[3, 3]])


### Some softer multiplications

In this case the q(uery) isn't as decisive as before, the q matrix is softer. In a numeric world the softer values are responsible for some weighted v(alues) matrix. As the keys stay the same, every value in the v(alues) matrix counts except the first, as the first value in the q(ueriy) matrix is zero. The other v(alues) are weighted.

In [30]:
q= torch.FloatTensor([0,0.3,0.2,0.4,0.1]).unsqueeze(0)
k = F.one_hot(torch.arange(5), 5).to(torch.float)
v = v.to(torch.float)

print("q")
print(q)
print("k")
print(k)
print("v")
print(v)

q_times_k=torch.mm(q,k.T)
print("q_times_k")
print(q_times_k)
print("q_times_k_times_v")
print(torch.mm(q_times_k, v))

q
tensor([[0.0000, 0.3000, 0.2000, 0.4000, 0.1000]])
k
tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]])
v
tensor([[7., 2.],
        [2., 5.],
        [3., 3.],
        [6., 2.],
        [7., 3.]])
q_times_k
tensor([[0.0000, 0.3000, 0.2000, 0.4000, 0.1000]])
q_times_k_times_v
tensor([[4.3000, 3.2000]])


### Softer keys

As the keys of matrices in deep learning most of the times aren't that explicitly defined, I have adjusted these too, slightly, to see some effect. I have adjusted the keys for the second v(alues) row. As a result these values changed, from [2,5] to [2.5,4.0]. This change has its effect on the final result of the complete matrix multiplication. 

In [31]:
q= torch.FloatTensor([0,0.3,0.2,0.4,0.1]).unsqueeze(0)
k = F.one_hot(torch.arange(5), 5).to(torch.float)
k[1,1]=0.5
k[2,1]=0.5
v = v.to(torch.float)

print("q")
print(q)
print("k")
print(k)
print("v")
print(v)

k_times_v = torch.mm(k.T, v)
print("k_times_data")
print(k_times_v)

q_times_k_times_v=torch.mm(q,k_times_v)
print("q_times_k_times_v")
print(q_times_k_times_v)


q
tensor([[0.0000, 0.3000, 0.2000, 0.4000, 0.1000]])
k
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5000, 1.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 1.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 1.0000]])
v
tensor([[7., 2.],
        [2., 5.],
        [3., 3.],
        [6., 2.],
        [7., 3.]])
k_times_data
tensor([[7.0000, 2.0000],
        [2.5000, 4.0000],
        [3.0000, 3.0000],
        [6.0000, 2.0000],
        [7.0000, 3.0000]])
q_times_k_times_v
tensor([[4.4500, 2.9000]])


### The attention usage

In this case attention is used to adjust word/token embeddings to express semantical or syntactical relations between different parts of a sentence. As I only use Attention once, it can only convey the information it is capable of conveying, as a sentence can be composed of multiple semantical and syntactical twists this is probably to limited for the texts it is trained on. It should provide better results than a model without attention.

In [51]:
class Attention(nn.Module):
    def __init__(self, d_in,d_out, n_layers):
        super().__init__()
        self.d_in=d_in
        self.d_out=d_out
        self.n_layers =n_layers
        self.Q=nn.Linear(self.n_layers*d_in,d_out)
        self.K=nn.Linear(d_in,d_out)
        self.V=nn.Linear(d_in,d_out)
    def forward(self,q ,x):
        queries=self.Q(q)
        keys = self.K(x)
        values = self.V(x)
        scores = torch.bmm(queries, keys.transpose(1,2))
        scores = scores/ (self.d_out**0.5)
        attention = F.softmax(scores,dim=2)
        hidden_states= torch.bmm(attention,values)
        return hidden_states

### The model with attention

As it is a bit nonsensical to apply attention to an LSTM block beign used to generate hidden and cell values while it is generating these values. A secondary LSTM block is used to absorb the results of applying attention to the values produced by the first LSTM block.

In [52]:
class LSTMForWordGenerationWithAttention(nn.Module):
    def __init__(self, word2idx, idx2word, embedding_dim=128, hidden_size=256, bidirectional=False, n_layers=3):
        super().__init__()
        self.word2idx = word2idx
        self.idx2word = idx2word
        self.embedding_dim = embedding_dim
        self.num_words = len(word2idx)
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.num_directions = 2 if bidirectional else 1

        self.embedding = nn.Embedding(self.num_words, embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim,
                            hidden_size=hidden_size,
                            num_layers=n_layers,
                            batch_first=True,
                            bidirectional=bidirectional)

        self.attention = Attention(self.embedding_dim, self.n_layers * self.num_directions * self.embedding_dim, self.n_layers)

        self.lstm2 = nn.LSTM(input_size=self.n_layers * self.num_directions * self.embedding_dim,
                             hidden_size=self.n_layers * self.num_directions * self.hidden_size,
                             num_layers=1,
                             batch_first=True,
                             bidirectional=False)

        self.fc = nn.Linear(self.n_layers * self.num_directions * self.hidden_size, self.num_words)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        batch_size = x.size(0)
        output_enc_dec = FloatTensor()
        for i in range(x.shape[1]):
            x_i = self.embedding(x[:, i])
            x_i = x_i.unsqueeze(1)

            if i == 0:
                out, (hidden, cell) = self.lstm(x_i)
            else:
                out, (hidden, cell) = self.lstm(x_i, (hidden, cell))

            hidden = hidden.reshape(1, -1, self.n_layers * self.num_directions * self.hidden_size)
            cell = cell.reshape(1, -1, self.n_layers * self.num_directions * self.hidden_size)

            attention_output = self.attention(out, x_i)

            out_lstm2, (h, c) = self.lstm2(attention_output, (hidden, cell))

            hidden = torch.div(torch.add(hidden, h), 2, rounding_mode=None)
            cell = torch.div(torch.add(cell, c), 2, rounding_mode=None)

            hidden = hidden.reshape(self.n_layers * self.num_directions, batch_size, -1)
            cell = cell.reshape(self.n_layers * self.num_directions, batch_size, -1)

            output_enc_dec = torch.cat([output_enc_dec, out_lstm2.unsqueeze(1)], dim=1)

        output_enc_dec = output_enc_dec.squeeze(2)
        logits = self.fc(output_enc_dec)
        return logits

    def write(self, text, max_words, greedy=False):
        idx = torch.tensor([self.word2idx[w] for w in text])
        hidden = Tensor()
        cell = Tensor()
        for i in range(max_words):
            if i == 0:
                selected_idx = idx
            else:
                selected_idx = idx[-1].unsqueeze(0)

            x = self.embedding(selected_idx)
            x = x.unsqueeze(0)

            if i == 0:
                out, (hidden, cell) = self.lstm(x)
            else:
                out, (hidden, cell) = self.lstm(x, (hidden, cell))

            hidden1 = hidden.detach().clone().reshape(1, 1, -1)
            cell1 = cell.detach().clone().reshape(1, 1, -1)

            attention_output = self.attention(out, x)
            out_lstm2, (h, c) = self.lstm2(attention_output, (hidden1, cell1))

            out = self.fc(out_lstm2)
            out = out.squeeze(0)

            if len(out) > 1:
                out = out[-1, :].unsqueeze(0)

            probs = self.softmax(out)

            if greedy:
                idx_next = torch.argmax(probs).squeeze(0)
            else:
                idx_next = torch.multinomial(probs, num_samples=1).squeeze(0)

            idx = torch.cat([idx, idx_next])

        gen_string = [self.idx2word[int(w)] for w in idx]
        gen_string = " ".join(gen_string)
        return gen_string


In [49]:
def get_config_attention():
    config={}
    config["iterations"]=300
    config["max_len"]=20
    config["evaluate_interval"]=30
    config["embedding_dim"]=128
    config["hidden_size"]=256
    config["n_layers"]=2
    config["lr"]=0.003
    config["batch_size"]=64
    config["bidirectional"]=True
    return config


In [53]:
from wakepy import keep 
config=get_config_attention()
model = LSTMForWordGenerationWithAttention(word2idx, idx2word, embedding_dim=config["embedding_dim"],
                                      hidden_size=config["hidden_size"], n_layers=config["n_layers"])
with keep.running():
    train(model, word2idx, idx2word, tokens, config)


--------------------------------------
training text length 20
Iteration 0
Loss 8.84449577331543
Spells staying inscription aconite lover lips glistening p-pick Bathilda sunk mightn cheating Scars pocket stories dressed hope Piers forest cut belting month Longbottom infusion price frantic Bleaaargh POTTER rooting mistake Arent golden-brown wizard sniffily runnin committing importantly families stroked MASTER Smarmy conducted dampen dived books CRACKPOT edged Tuesday ribs woke Walking
--------------------------------------
--------------------------------------
training text length 20
Iteration 30
Loss 6.248763084411621
Spells juicy thieves Hut-on-the-Rock directing bathrobe the huge , Ive of the mention him got year their what of Twentieth going do Catch Im guess knickerbockers during it , run into the Halloween like ! cleared every painful red had a Lets , though and tricked wonderful be lullaby Hagrid in
--------------------------------------
--------------------------------------
tr